# Claim 1

In [21]:
'''
pandas will be our data manipulation module
'''
import pandas as pd
pd.set_option('display.max_columns', None, 'display.max_rows', 200)

'''
numpy will be our array computing module
'''
import numpy as np

'''
tqdm allows us to easily add progress bars to our processes
'''
from tqdm import tqdm

'''
display will allow us to easily display custom data types like dataframes
'''
from IPython.display import display

'''
built-in python modules
'''
import os
import string

# Loading Dataframes

In [22]:
# organizations2 = pd.read_csv('./orgs_fundingRounds_merged.csv')
# #organizations2 = organizations2.drop(columns=['country_code', 'state_code', 'status',
# #                                            'category_groups_list', 'num_funding_rounds', 'primary_role'])
# organizations2

In [23]:
# org_names = organizations2['name']
# organizations2[org_names.isin(org_names[org_names.duplicated()])].sort_values(by='name', axis=0)

In [24]:
organizations = pd.read_csv('./organizations_filtered.csv')
# organizations = organizations.drop(columns=['country_code', 'state_code', 'status',
#                                             'category_groups_list', 'num_funding_rounds', 'primary_role'])
organizations = organizations.drop(columns=['country_code', 'state_code', 'category_groups_list', 'primary_role'])
organizations.dropna(subset=['founded_on'])
organizations

,uuid,name,status,category_list,num_funding_rounds,total_funding_usd,founded_on,last_funding_on
0,74a20af3-f4dd-6188-de60-c4ee6cd0ca4a,Ant Group,operating,"Banking,Financial Services,FinTech,Payments",4.0,2.200000e+10,2014-10-01,2018-06-08
1,ba2dfa91-ce6d-2347-3b74-4ffdffa1b7ee,Gazprom,ipo,"Business Intelligence,Energy,Energy Efficiency...",3.0,1.476479e+10,1989-01-01,2019-12-24
2,47f9688f-00a9-23f9-3a64-179bc6dd31d4,ByteDance,operating,"Artificial Intelligence,Content,Data Mining,Ma...",12.0,9.405000e+09,2012-03-01,2020-12-11
3,416156d2-f0de-7c42-8303-6eaeab697c26,Robinhood,operating,"Cryptocurrency,Financial Services,FinTech,Stoc...",23.0,5.574325e+09,2013-01-01,2021-02-01
4,fc254a10-f558-2813-9435-45c34bd6ec3b,Epic Games,operating,"Developer Platform,Gaming,Software,Video Games",8.0,5.110000e+09,1991-01-01,2021-04-13
...,...,...,...,...,...,...,...,...
37251,3947e7b5-8a6b-4fb1-bd7f-d8f5507401d4,Get Help Tax and Bookkeeping,operating,"Accounting,Consulting,Finance,Financial Servic...",NaN,NaN,NaN,NaN
37252,ee15f0de-46a3-49a7-9ddd-d9738bb92f84,ottomon,operating,"Augmented Reality,Internet of Things",NaN,NaN,2017-12-27,NaN
37253,2708485a-8af1-4f77-a244-597603ace22d,Mitivate,operating,"Analytics,Health Care,Machine Learning",NaN,NaN,NaN,NaN
37254,1773734f-75b9-448f-8e25-25ac9011ee98,Futura VR Studio,operating,"Electronics,Information Technology,Video,Video...",NaN,NaN,2016-01-01,NaN


# Planning Data Structures

In [25]:
last_funded_list = organizations.founded_on.tolist()
distinct_years = []

for i in last_funded_list:
    year = str(i)[:4]
    if year not in distinct_years and year != 'nan':
        distinct_years.append(year)

distinct_years.sort()

print(f'There are {len(distinct_years)} distinct years.')
print(f'The min year is {min(distinct_years)}.')
print(f'The max year is {max(distinct_years)}.')

There are 181 distinct years.
The min year is 1472.
The max year is 2021.


Despite the large range of years in the founded_on column, we will only use 2016-2020, since the growth in the years prior is irrelevant in predicting the future success of a vertical. We will not be using new companies in 2021, since the year is not over and therefore the data is incomplete.

In [26]:
years = [*range(2016, 2021, 1)]
organizations = organizations.loc[ (organizations['founded_on'] > '2015-12-31') & (organizations['founded_on'] < '2021-01-01') ]
organizations = organizations.reset_index(drop=True)

We will create a dictionary in the format {'vertical': {1983: company_count, 1984: company_count...}}

In [27]:
verticals = ['Machine Learning', 'FinTech', 'Artificial Intelligence', 'Video Games',
            'Renewable Energy', 'Cyber Security', 'Social Network', 'Virtual Reality',
            'Cryptocurrency', 'Augmented Reality']

vertical_growth = {}

for i in verticals:
    vertical_growth[i] = {}

for i in vertical_growth:
    for j in years:
        vertical_growth[i][j] = 0

Now, we just need to iterate through the dataframe and update the year counts.

# Updating Year Counts per Vertical

Process:
- iterate through every entry in the dataframe
    - iterate through every vertical in verticals
        - check if category_list contains vertical
            - grab the year off of founded_on and increment the count (vertical_growth[vertical][year] += 1)


In [28]:
for i in organizations.index.values:
    for j in verticals:
        if j in str( organizations.iloc[i]['category_list'] ).split(','):
            if str( organizations.iloc[i]['founded_on'] )[:4] != 'nan':
                vertical_growth[ j ][ int( str( organizations.iloc[i]['founded_on'] )[:4] ) ] += 1

Next, we get rid of years with count 0.

In [29]:
for key, value in vertical_growth.items():
    for k in list(value.keys()):
        if value[k] == 0:
            del vertical_growth[key][k]

In [30]:
vertical_growth

{'Machine Learning': {2016: 967, 2017: 833, 2018: 190, 2019: 22, 2020: 5},
 'FinTech': {2016: 1116, 2017: 923, 2018: 244, 2019: 34, 2020: 12},
 'Artificial Intelligence': {2016: 1549,
  2017: 1376,
  2018: 311,
  2019: 42,
  2020: 10},
 'Video Games': {2016: 181, 2017: 114, 2018: 34, 2019: 6, 2020: 1},
 'Renewable Energy': {2016: 136, 2017: 84, 2018: 20, 2019: 3, 2020: 2},
 'Cyber Security': {2016: 357, 2017: 265, 2018: 63, 2019: 8, 2020: 9},
 'Social Network': {2016: 42, 2017: 20, 2018: 6, 2019: 1},
 'Virtual Reality': {2016: 418, 2017: 211, 2018: 41, 2019: 4, 2020: 2},
 'Cryptocurrency': {2016: 168, 2017: 515, 2018: 168, 2019: 6},
 'Augmented Reality': {2016: 239, 2017: 156, 2018: 37, 2019: 3, 2020: 1}}

In [31]:
pd.DataFrame.from_dict(vertical_growth)

,Machine Learning,FinTech,Artificial Intelligence,Video Games,Renewable Energy,Cyber Security,Social Network,Virtual Reality,Cryptocurrency,Augmented Reality
2016,967,1116,1549,181,136,357,42.0,418,168.0,239
2017,833,923,1376,114,84,265,20.0,211,515.0,156
2018,190,244,311,34,20,63,6.0,41,168.0,37
2019,22,34,42,6,3,8,1.0,4,6.0,3
2020,5,12,10,1,2,9,NaN,2,NaN,1


In [32]:
cyber_security = organizations.loc[ organizations['category_list'].str.contains('Cyber Security') ]
cyber_security = cyber_security.loc[ cyber_security['founded_on'].str.contains('2018')]
cyber_security

,uuid,name,status,category_list,num_funding_rounds,total_funding_usd,founded_on,last_funding_on
161,ad22a2a6-015d-4ff1-9516-f517cedebeee,Perimeter 81,operating,"Cloud Security,Cyber Security,Network Security...",5.0,65000000.0,2018-01-01,2020-08-11
299,18b59b98-9e94-4b72-a5fd-f534217a266c,Vulcan Cyber,operating,"Cyber Security,Network Security,Risk Managemen...",3.0,35000000.0,2018-03-18,2021-03-17
343,21f7a09e-21f8-4bec-a44d-8ead9b0420c4,Origo,operating,"Blockchain,Cyber Security,Privacy,Security",1.0,30000000.0,2018-01-01,2018-06-10
370,bbec44b9-06a5-4227-9110-2e97f1b99b27,Uppsala Security,operating,"Blockchain,Cyber Security,Information Technology",1.0,27700000.0,2018-01-11,2018-06-19
387,c69ec9f3-5186-47c0-8f8f-74f0bcf0d76a,Valtix,operating,"Cloud Security,Cyber Security,Network Security...",2.0,26500000.0,2018-03-01,2021-01-21
792,0fd08072-e212-4e62-b6af-a5d9d2f24c9a,Altitude Networks,operating,"Cloud Security,Collaboration,Cyber Security,In...",3.0,11000000.0,2018-04-01,2020-02-06
975,59e8b734-dd39-4d43-ab13-cb41fe6cb663,Inspectiv,operating,"Cyber Security,Information Technology,Internet...",2.0,7500000.0,2018-06-01,2020-11-16
1043,ee1fd729-a4eb-e633-d41d-44c4de681131,Strattic,operating,"Cloud Infrastructure,CMS,Cyber Security,Inform...",3.0,6500000.0,2018-01-30,2020-02-26
1302,17ddf8f4-bd62-45ad-3307-e16078153977,Kindite,acquired,"Big Data,Cloud Security,Cyber Security,Enterpr...",2.0,4000000.0,2018-05-01,2018-10-16
1320,1139e8ac-4b37-4fad-8158-c91ce74285a5,Magic,operating,"Apps,Cyber Security,Developer Tools,Identity M...",1.0,4000000.0,2018-01-01,2020-05-29


In [34]:
raw_orgs = pd.read_csv('../dwx_data/organizations.csv')
raw_orgs

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,legal_name,roles,domain,homepage_url,country_code,state_code,region,city,address,postal_code,status,short_description,category_list,category_groups_list,num_funding_rounds,total_funding_usd,total_funding,total_funding_currency_code,founded_on,last_funding_on,closed_on,employee_count,email,phone,facebook_url,linkedin_url,twitter_url,logo_url,alias1,alias2,alias3,primary_role,num_exits
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,wetpaint,https://www.crunchbase.com/organization/wetpaint,141336.0,2007-05-25 13:51:27,2019-06-24 22:19:25,NaN,company,wetpaint.com,http://www.wetpaint.com/,USA,NY,New York,New York,902 Broadway 11th Floor New,10010,acquired,Wetpaint offers an online social publishing pl...,"Publishing,Social Media,Social Media Management","Content and Publishing,Internet Services,Media...",3.0,3.975000e+07,3.975000e+07,USD,2005-06-01,2008-05-19,NaN,51-100,info@wetpaint.com,206-859-6300,https://www.facebook.com/Wetpaint,https://www.linkedin.com/company/wetpaint,https://twitter.com/wetpainttv,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,NaN
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,6313.0,2007-05-26 02:30:28,2018-10-27 00:29:49,NaN,"investor,company",zoho.com,https://www.zoho.com/,USA,CA,California,Pleasanton,4141 Hacienda Drive,94588,operating,"Zoho offers a suite of business, collaboration...","Cloud Computing,Collaboration,CRM,Developer To...","Administrative Services,Information Technology...",NaN,NaN,NaN,NaN,1996-09-15,NaN,NaN,1001-5000,info@zohocorp.com,NaN,http://www.facebook.com/zoho,http://www.linkedin.com/company/zoho-corporati...,http://twitter.com/zoho,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,1.0
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,digg,https://www.crunchbase.com/organization/digg,8697.0,2007-05-26 03:03:23,2018-12-10 10:09:14,"Digg Holdings, LLC",company,digg.com,http://www.digg.com,USA,NY,New York,New York,NaN,NaN,acquired,Digg Inc. operates a website that enables its ...,"Internet,Social Media,Social Network","Internet Services,Media and Entertainment",6.0,4.900000e+07,4.900000e+07,USD,2004-10-11,2016-09-13,NaN,51-100,feedback@digg.com,877-342-7222,http://www.facebook.com/digg,http://www.linkedin.com/company/digg,http://twitter.com/digg,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,NaN
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,omidyar-network,https://www.crunchbase.com/organization/omidya...,79785.0,2007-05-26 03:21:34,2019-06-19 12:17:48,NaN,investor,omidyar.com,http://www.omidyar.com,USA,CA,California,Redwood City,1991 Broadway Suite 200,94063,operating,Omidyar Network is an investment firm.,"Enterprise Software,Financial Services,Venture...","Financial Services,Lending and Investments,Sof...",NaN,NaN,NaN,NaN,2004-01-01,NaN,NaN,101-250,info@omidyar.com,650.482.2500,http://www.facebook.com/OmidyarNetwork,http://www.linkedin.com/company/22806,http://twitter.com/OmidyarNetwork,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,investor,38.0
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,organization,facebook,https://www.crunchbase.com/organization/facebook,22.0,2007-05-26 04:22:15,2021-05-17 14:06:02,"Facebook, Inc.","investor,company",facebook.com,http://www.facebook.com,USA,CA,California,Menlo Park,1 Hacker Way,94025,ipo,Facebook is an online social networking servic...,"Mobile Apps,Photo Sharing,Social Media,Social ...","Apps,Content and Publishing,Internet Services,...",17.0,1.612282e+10,1.612282e+10,USD,2004-02-04,2014-10-20,NaN,10000+,NaN,NaN,https://www.facebook.com/facebook,http://www.linkedin.com/company/facebook,https://twitter.com/facebook,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [37]:
p_and_s = raw_orgs.loc[ raw_orgs['category_groups_list'].str.contains('Privacy and Security', na=False) ]

p_and_s

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,legal_name,roles,domain,homepage_url,country_code,state_code,region,city,address,postal_code,status,short_description,category_list,category_groups_list,num_funding_rounds,total_funding_usd,total_funding,total_funding_currency_code,founded_on,last_funding_on,closed_on,employee_count,email,phone,facebook_url,linkedin_url,twitter_url,logo_url,alias1,alias2,alias3,primary_role,num_exits
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,6313.0,2007-05-26 02:30:28,2018-10-27 00:29:49,NaN,"investor,company",zoho.com,https://www.zoho.com/,USA,CA,California,Pleasanton,4141 Hacienda Drive,94588,operating,"Zoho offers a suite of business, collaboration...","Cloud Computing,Collaboration,CRM,Developer To...","Administrative Services,Information Technology...",NaN,NaN,NaN,NaN,1996-09-15,NaN,NaN,1001-5000,info@zohocorp.com,NaN,http://www.facebook.com/zoho,http://www.linkedin.com/company/zoho-corporati...,http://twitter.com/zoho,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,1.0
29,8cb7f859-e757-0937-a18b-e7992c167203,Postini,organization,postini,https://www.crunchbase.com/organization/postini,165984.0,2007-06-08 19:19:51,2020-10-19 05:12:07,NaN,company,postini.com,http://postini.com,USA,CA,California,San Carlos,959 Skyway Road Suite 200,94070,acquired,Postini offers solutions that protect companie...,"Cyber Security,Internet,Security","Information Technology,Internet Services,Priva...",5.0,26000000.0,26000000.0,USD,1999-06-02,2003-09-15,NaN,251-500,NaN,NaN,NaN,https://www.linkedin.com/company/postini/about/,https://www.twitter.com/gsuite,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,NaN
66,b93d25ff-cf38-929e-57a6-b40bf77db9ca,LifeLock,organization,lifelock,https://www.crunchbase.com/organization/lifelock,11067.0,2007-06-18 18:31:31,2018-12-07 06:42:54,"LifeLock, Inc.",company,lifelock.com,http://www.lifelock.com,USA,AZ,Arizona,Tempe,60 E. Rio Salado Pkwy Suite 400,85281,acquired,LifeLock is an identity theft protection compa...,"Consumer,Security,Service Industry","Other,Privacy and Security",7.0,201340000.0,201340000.0,USD,2005-08-01,2016-05-21,NaN,501-1000,member.feedback@lifelock.com,800-416-0599,http://www.facebook.com/LifeLock,http://www.linkedin.com/company/lifelock,http://twitter.com/LifeLock,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,NaN
160,bca6051d-bfab-c24b-bc0b-9ecaabf35feb,Haute Secure,organization,hautesecure,https://www.crunchbase.com/organization/hautes...,257274.0,2007-07-11 17:57:10,2020-10-01 07:01:47,NaN,company,hautesecure.com,http://hautesecure.com,USA,WA,Washington,Seattle,2212 Queen Anne Ave. N #282,98109,acquired,Haute Secure is a security company providing c...,"Security,Software","Privacy and Security,Software",2.0,2100000.0,2100000.0,USD,2006-01-01,2009-03-09,NaN,1-10,info@hautesecure.com,NaN,NaN,NaN,NaN,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,NaN
213,c7102390-0901-e1d7-1805-a1dabc40f6ca,LifeShield,organization,ingrid,https://www.crunchbase.com/organization/ingrid,57995.0,2007-07-17 15:24:58,2020-12-10 04:57:05,NaN,company,lifeshield.com,http://www.lifeshield.com,USA,PA,Pennsylvania,Langhorne,2021 Cabot Blvd W,19047,acquired,LifeShield is a monitored wireless security sy...,"Digital Entertainment,Network Security,Physica...","Administrative Services,Hardware,Information T...",6.0,43050000.0,43050000.0,USD,2004-01-01,2011-07-20,NaN,51-100,support@lifeshield.com,484-645-1455,http://www.facebook.com/wirelessalarmsystem,https://www.linkedin.com/company/lifeshield/,https://www.twitter.com/lshomesecurity,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377971,c3805cba-9f26-41eb-89f1-fb074b399aa2,Sigma Science,o